In [212]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
import networkx as nx
import collections
import cProfile

Notes:  
Would lots of transactions in short succession be considered suspicious??  See sender id 2.

In [218]:
"""Read file. Return dataframe.  Last field can have
embedded commas"""
def read_file(path):
    with open(os.path.join(path),'r') as f:
        header = [item.strip() for item in next(f).split(',')]
        lines = (line.split(',',len(header)-1) for line in f)
        df = pd.DataFrame(lines,columns=header,)
    return df

#This method keeps '/n' at the end of each line... Not an issue now
#because we'll drop that column, but something to note.

In [204]:
batch_data = read_file(os.path.join("paymo_input","batch_payment.csv"))

In [205]:
payment_history = batch_data[['id1','id2']][batch_data.id1.str.strip().str.isnumeric() & batch_data.id2.str.strip().str.isnumeric()]
payment_history = payment_history.apply(pd.to_numeric)

In [206]:
connected = payment_history.drop_duplicates()
#need to deal with mirrored pairs??
#consider set of sets??

###### Questions: 
* Need to remove mirrored pairs?  Could turn rows into a set (see frozenset)


In [207]:
g = nx.from_pandas_dataframe(connected,'id2','id1')

### Feature 2

In [210]:
stream_data = read_file(os.path.join("paymo_input","stream_payment.csv"))

In [228]:
%%timeit
test_set = stream_data[:100]
#with open(os.path.join("paymo_output","feature1.txt"),'w') as outfile:
d = {}#collections.OrderedDict()
 
for trans in test_set.itertuples():
    try:
        sender_id = int(trans.id1)
        recipient_id = int(trans.id2)
        #datetime.strptime(trans.time, '%Y-%m-%d %H:%M:%S')
    except (ValueError, TypeError) as err:
        d[trans.Index] = 'reject'
        continue
    try:
        if nx.astar_path_length(g,sender_id,recipient_id) <= 2:
            d[trans.Index] = 'verified'
        else:
            d[trans.Index] = 'unverified'
    except nx.NetworkXNoPath:
        d[trans.Index] = 'unverified'


1 loop, best of 3: 14 s per loop


In [225]:
d

{0: 'verified',
 1: 'verified',
 2: 'verified',
 3: 'verified',
 4: 'verified',
 5: 'unverified',
 6: 'verified',
 7: 'unverified',
 8: 'verified',
 9: 'verified'}

In [175]:
myDict =  {key: value for (key, value) in d.items() if value != 'verified' }

In [184]:
myDict.keys()

dict_keys([1078208, 2862725, 1078209, 3765064, 411570, 3765062, 3765063, 3772588, 411565, 411566, 411567, 411568, 411569, 2726322, 3765061, 1705673, 620092, 620093, 620094, 620095, 620096, 620097, 3765065, 1299012, 1299013, 1299014, 1299015, 1299016, 1299017, 1705674, 1705675, 1705676, 1705677, 1705678, 3936062, 3765066, 3753051, 3753052, 3753053, 3753054, 3753055, 3753056, 1078206, 1078204, 1078205, 3080304, 3080305, 3080306, 3080307, 3080308, 3080309, 1078207])

In [190]:
stream_data.ix[myDict.keys()]


,time,id1,id2,amount,message
1078208,Susan Paid for Claires contacts $220 the 70/3...,None,None,None,None
2862725,German-Austria must be restored to the great G...,no. Even if the union were a matter of econom...,and even if it were to be disadvantageous fro...,still it ought to take place. People of the s...,"only then can the moral right arise, from the..."
1078209,AMOUNT TO PAID TO SUSAN $355.41 11/6/...,None,None,None,None
3765064,SUBTOTAL $165.00\n,None,None,None,None
411570,AMOUNT TO PAID TO SUSAN $355.41 11/6/...,None,None,None,None
3765062,CCP Cheer Shoes paid in August - 80.00 $40.00\n,None,None,None,None
3765063,CCP Frequency paid in August - 215.00 $107....,None,None,None,None
3772588,German-Austria must be restored to the great G...,no. Even if the union were a matter of econom...,and even if it were to be disadvantageous fro...,still it ought to take place. People of the s...,"only then can the moral right arise, from the..."
411565,JP HOSA Dues paid in August - 35.00 $17.50\n,None,None,None,None
411566,CCP Cheer Shoes paid in August - 80.00 $40.00\n,None,None,None,None


##### Note:
Am doing itertuples to simulate stream.  But could do this as pandas operation with lambda....


In [231]:
def feature2_status(trans):
    try:
        datetime.strptime(trans.time, '%Y-%m-%d %H:%M:%S')
        sender_id = int(trans.id1)
        recipient_id = int(trans.id2)
    except (ValueError, TypeError) as err:
        return 'invalid'
    try:
        if nx.astar_path_length(g,sender_id,recipient_id) <= 2:
            return 'verified'
        else:
            return 'unverified'
    except nx.NetworkXNoPath:
        return 'unverified'
    return 'Error'
   
    

In [ ]:
%%timeit
results = test_set.apply(lambda x: feature2_status(x),axis=1)